In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-3-2/'

In [3]:
page1 = requests.get(url)

In [4]:
soup = BeautifulSoup(page1.content, 'html.parser')

In [5]:
article_title = soup.find('h1').text

In [6]:
article_title

'Rise of telemedicine and its Impact on Livelihood by 2040'

In [7]:
text = soup.find("div", class_="td-post-content tagdiv-type")

In [8]:
paragraphs = text.findAll('p')
article_text = "\n".join([p.get_text() for p in paragraphs]).replace('\n', ' ')
    
# article_text    

In [9]:
df = pd.read_csv('D://Intern_FraudDetection_ML//DataSets//input_data.csv')

In [10]:
df.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [11]:
df.shape

(114, 2)

In [16]:
!pip install nltk

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.5 MB 1.9 MB/s eta 0:00:01
     --- ------------------------------------ 0.1/1.5 MB 1.8 MB/s eta 0:00:01
     ------ --------------------------------- 0.3/1.5 MB 2.2 MB/s eta 0:00:01
     ---------- ----------------------------- 0.4/1.5 MB 2.3 MB/s eta 0:00:01
     -------------------- ------------------- 0.8/1.5 MB 3.3 MB/s eta 0:00:01
     -------------------------------- ------- 1.2/1.5 MB 4.4 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 4.6 MB/s eta 0:00:00
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/e6/7c/96a44dabe8577f43ac34e34d0ac098ee42390a06fee4cbe8b5317ecf2520/regex-2023.8.8-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.0 MB/s eta 0:00:00
  Obtaining de

In [17]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [31]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [21]:
def load_word_list(file_path):
    with open(file_path, "r") as file:
        return [word.strip() for word in file.readlines()]
    
stopwords_files = [ 
    "D://Intern_FraudDetection_ML//DataSets//StopWords_Currencies.txt",
    "D://Intern_FraudDetection_ML//DataSets//StopWords_Geographic.txt",
    "D://Intern_FraudDetection_ML//DataSets//StopWords_Names.txt",
    "D://Intern_FraudDetection_ML//DataSets//StopWords_DatesandNumbers.txt",
    "D://Intern_FraudDetection_ML//DataSets//StopWords_Generic.txt",
    "D://Intern_FraudDetection_ML//DataSets//StopWords_GenericLong.txt",
    "D://Intern_FraudDetection_ML//DataSets//StopWords_Auditor.txt"
]    

combined_stopwords = set()
for file_path in stopwords_files:
    combined_stopwords.update(load_word_list(file_path))

In [22]:
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...


In [23]:
positive_words_list = 'D://Intern_FraudDetection_ML//DataSets//positive-words.txt'
negative_words_list = 'D://Intern_FraudDetection_ML//DataSets//negative-words.txt'

positive_words = load_word_list(positive_words_list)
negative_words = load_word_list(negative_words_list)

In [36]:
!pip install textstat
import textstat

     ---------------------------------------- 0.0/105.1 kB ? eta -:--:--
     ----------- ---------------------------- 30.7/105.1 kB ? eta -:--:--
     -------------------------------------  102.4/105.1 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 105.1/105.1 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
     -- ------------------------------------- 0.1/2.0 MB 2.4 MB/s eta 0:00:01
     ----- ---------------------------------- 0.3/2.0 MB 2.9 MB/s eta 0:00:01
     ---------- ----------------------------- 0.5/2.0 MB 3.7 MB/s eta 0:00:01
     --------------------- ------------------ 1.0/2.0 MB 5.5 MB/s eta 0:00:01
     -------------------------------------- - 1.9/2.0 MB 8.6 MB/s eta 0:00:01
     ---------------------------------------- 2.0/2.0 MB 7.4 MB/s eta 0:00:00


In [40]:
output_data = []

for index, row in df.iterrows():
    url_id = row["URL_ID"]
    url = row["URL"]   
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    title = soup.find('h1')
    if title:
        article_title = title.text   

    text = soup.find("div", class_="td-post-content tagdiv-type")    
    if text:
        paragraphs = text.findAll('p')
        article_text = "\n".join([p.get_text() for p in paragraphs]).replace('\n', ' ')

    if article_title and article_text:
        output_filename = "{}.txt".format(url_id)
        with open(output_filename, "w", encoding="utf-8") as output_file:
            output_file.write(article_title + '\n')
            output_file.write(article_text)
        
    words = nltk.word_tokenize(article_text.lower())
    words = [word for word in words if word not in combined_stopwords]
    
    positive_word_count = sum(1 for word in words if word in positive_words)
    negative_word_count = sum(1 for word in words if word in negative_words)
    
    total_word_count = len(words)
    if total_word_count > 0:
        positive_score = positive_word_count / total_word_count
        negative_score = negative_word_count / total_word_count
    else:
        positive_score = 0.0
        negative_score = 0.0
        
    neutral_score = 1 - (positive_score + negative_score)    
    polarity_score = positive_score - negative_score
    subjectivity_score = positive_score + negative_score + neutral_score
    
    sentences = nltk.sent_tokenize(article_text)
    total_words = len(nltk.word_tokenize(article_text))
    total_syllables = sum([textstat.syllable_count(word) for word in nltk.word_tokenize(article_text)])
    
    avg_sentence_length = total_words / len(sentences)
    percentage_complex_words = (textstat.difficult_words(article_text) / total_words) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    
    avg_words_per_sentence = total_words / len(sentences)
    complex_word_count = textstat.difficult_words(article_text)
    syllables_per_word = total_syllables / total_words
    personal_pronouns = article_text.count("I") + article_text.count("me") + article_text.count("my") + article_text.count("mine")
    avg_word_length = total_syllables / total_words
    
    article_data = {
            "URL_ID": url_id,
            "URL": url,
            "POSITIVE SCORE": positive_score,
            "NEGATIVE SCORE": negative_score,
            "POLARITY SCORE": polarity_score,
            "SUBJECTIVITY SCORE": subjectivity_score,
            "AVG SENTENCE LENGTH": avg_sentence_length,
            "PERCENTAGE OF COMPLEX WORDS": percentage_complex_words,
            "FOG INDEX": fog_index,
            "AVG NUMBER OF WORDS PER SENTENCE": avg_words_per_sentence,
            "COMPLEX WORD COUNT": complex_word_count,
            "WORD COUNT": total_words,
            "SYLLABLE PER WORD": syllables_per_word,
            "PERSONAL PRONOUNS": personal_pronouns,
            "AVG WORD LENGTH": avg_word_length
        }
    output_data.append(article_data)
    
output_df = pd.DataFrame(output_data)    

output_file = "output_data.csv"
output_df.to_csv(output_file, index=False)

print("Output saved to:", output_file)

Output saved to: output_data.csv
